# Modelagem #


In [15]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score


df1 = pd.read_csv('Dados_Sem.csv', index_col=0)
df2 = pd.read_csv('Dados_Com.csv', index_col=0)


In [16]:
df2['order_date'] = pd.to_datetime(df2['order_date'])
#df_date = df.copy()

# Criando colunas para facilitar análises
df2['minute'] = df2['order_date'].dt.minute
df2['hour'] = df2['order_date'].dt.hour
df2['day'] = df2['order_date'].dt.day
df2['month'] = df2['order_date'].dt.month
df2['year'] = df2['order_date'].dt.year
df2['weekday'] = df2['order_date'].dt.weekday # Domingo é 0

In [17]:
# Definir as variáveis independentes (características)
X = df2[['hub_name', 'hub_state', 'store_name', 'hour', 'weekday', 'delivery_distance_meters', 'payment_method', 'channel_name', 'channel_type', 'order_delivery_fee']] 
X_encoded = pd.get_dummies(X, columns=['hub_name', 'hub_state', 'store_name', 'payment_method', 'channel_name', 'channel_type'])
X_not_encoded =df2[['hour', 'weekday', 'delivery_distance_meters','order_delivery_fee']]

# Definir a variável dependente (o que você quer prever)
y = df2['order_metric_cycle_time']

# Dividir os dados em conjuntos de treino e teste para variáveis codificadas
X_train_encoded, X_test_encoded, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Supondo que X_not_encoded são as variáveis que não passaram pelo Dummy Encoding
X_train_not_encoded, X_test_not_encoded, _, _ = train_test_split(X_not_encoded, y, test_size=0.2, random_state=42)

# Concatenar as colunas das variáveis codificadas e não codificadas
X_train = pd.concat([X_train_encoded, X_train_not_encoded], axis=1)
X_test = pd.concat([X_test_encoded, X_test_not_encoded], axis=1)

# Instanciar o modelo de regressão linear
model = LinearRegression()

# Treinar o modelo
model.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
predictions = model.predict(X_test)

# Avaliar o desempenho do modelo
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
print(f'MSE (Mean Squared Error): {mse}')
print(f'RMSE (Root Mean Squared Error): {rmse}')

MSE (Mean Squared Error): 3992.012079973804
RMSE (Root Mean Squared Error): 63.182371591875246


In [19]:
# Criando faixas para categorizar o tempo de ciclo
df2['cycle_time_category'] = pd.cut(df2['order_metric_cycle_time'], bins=3, labels=['curto', 'médio', 'longo'])

# Definir as variáveis independentes (características)
X = df2[['hub_name', 'hub_state', 'store_name', 'hour', 'weekday', 'delivery_distance_meters', 'payment_method', 'channel_name', 'channel_type', 'order_delivery_fee']] 
X_encoded = pd.get_dummies(X, columns=['hub_name', 'hub_state', 'store_name', 'payment_method', 'channel_name', 'channel_type'])
X_not_encoded = df2[['hour', 'weekday', 'delivery_distance_meters', 'order_delivery_fee']]

# Definir a variável dependente (o que você quer prever)
y = df2['cycle_time_category']  # Usar a categoria de tempo de ciclo

# Dividir os dados em conjuntos de treino e teste
X_train_encoded, X_test_encoded, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)
X_train_not_encoded, X_test_not_encoded, _, _ = train_test_split(X_not_encoded, y, test_size=0.2, random_state=42)

# Concatenar as colunas das variáveis codificadas e não codificadas
X_train = pd.concat([X_train_encoded, X_train_not_encoded], axis=1)
X_test = pd.concat([X_test_encoded, X_test_not_encoded], axis=1)

# Instanciar o modelo de regressão logística
model = LogisticRegression()

# Treinar o modelo
model.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
predictions = model.predict(X_test)

# Calcular a acurácia do modelo
accuracy = accuracy_score(y_test, predictions)
print(f'Acurácia: {accuracy}')

# Calcular a matriz de confusão
conf_matrix = confusion_matrix(y_test, predictions)
print(f'Matriz de Confusão:\n{conf_matrix}')

# Calcular a precisão para cada classe
precision = precision_score(y_test, predictions, average=None, labels=['curto', 'médio', 'longo'])
print(f'Precisão por classe:\n{precision}')

# Calcular o recall para cada classe
recall = recall_score(y_test, predictions, average=None, labels=['curto', 'médio', 'longo'])
print(f'Recall por classe:\n{recall}')

# Calcular o F1-score para cada classe
f1 = f1_score(y_test, predictions, average=None, labels=['curto', 'médio', 'longo'])
print(f'F1-score por classe:\n{f1}')

c:\Users\amand\Anaconda\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Acurácia: 0.9269615470104468
Matriz de Confusão:
[[62556     0     0]
 [   96     0     0]
 [ 4833     0     0]]


c:\Users\amand\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precisão por classe:
[0.92696155 0.         0.        ]
Recall por classe:
[1. 0. 0.]
F1-score por classe:
[0.96209657 0.         0.        ]


Preve bem 92-100% de cycle menores, ciclos medios e longos preve 0%
Sera que aqui entre o fato que normalmente ciclos longos sao outliers?